In [ ]:
# default_exp modelling

# Modelling meter readings

> First building a base model predicting medians/means, then linear, tree-based and ANN-based models.

Finding: make sure your test set values are not out of domain $\Rightarrow$ `timestampYear` in this notebook is put into the training set but there only takes on the value 2016.0, but in the test set it's 2017.0 and 2018.0, causing the predictions to zero out everywhere.

In [ ]:
#export
import pandas as pd
from pathlib import Path
import os
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import typing
import pickle


from sklearn import linear_model, tree, model_selection, ensemble

from fastai.tabular.all import *

import ipywidgets as widgets

In [ ]:
predict_test = False

In [ ]:
pd.options.plotting.backend = "plotly"

In [ ]:
base_path = Path("../data")

In [ ]:
#export
def evaluate_torch(y_true:torch.Tensor, y_pred:torch.Tensor): return torch.sqrt(F.mse_loss(y_true, y_pred))

## Loading

In [ ]:
%%time
with open(base_path/'var_types.pckl', 'rb') as f:
    var_types = pickle.load(f)

In [ ]:
continuous, categorical = var_types['cont'], var_types['cat']
continuous, categorical

In [ ]:
continuous = continuous[:9] + [continuous[-1]]
continuous

## Loading `X` and `X_test`

In [ ]:
%%time
X_all = pd.read_parquet(base_path/'X.parquet') #.sample(100000)

In [ ]:
X_all.groupby('building_id').size().describe()

In [ ]:
n_sample_per_bid = 1000
replace = True

In [ ]:
X = (X_all.groupby(['building_id', 'meter'])
     .sample(n=n_sample_per_bid, replace=replace))

In [ ]:
print(f'using {len(X)} samples = {len(X)/len(X_all)*100:.2f} %')

In [ ]:
display(len(X), X.head())

In [ ]:
X.groupby(['building_id', 'meter']).size().sort_values()

In [ ]:
%%time
if predict_test:
    X_test = pd.read_parquet(base_path/'X_test.parquet') #.sample(10000)

In [ ]:
if predict_test:
    display(len(X_test), X_test.head())

In [ ]:
#export
def split_dataset(X:pd.DataFrame, split_kind:str='random',
                  train_frac:float=8):
    
    def random_split():
        n_train = int(len(X)*train_frac)
        train_bool = X.index.isin(np.random.choice(X.index.values, size=n_train, replace=False))
        return train_bool
    
    def time_split():
        time_col = 'timestampElapsed'
        ts = X[time_col].sort_values(ascending=True)
        ix = int(len(X)*train_frac)
        threshold_t = ts.iloc[ix:].values[0]
        return X[time_col] < threshold_t
     
    split_funs = {
        'random': random_split,
        'time': time_split,
    }
    
    assert split_kind in split_funs
    train_bool = split_funs[split_kind]()
    
    train_idx = np.where(train_bool)[0]
    valid_idx = np.where(~train_bool)[0]

    return (list(train_idx), list(valid_idx))

In [ ]:
%%time
split_kind = 'random'
#split_kind = 'time'
splits = split_dataset(X, split_kind=split_kind, train_frac=.8)
#splits=None

In [ ]:
sorted(X.iloc[splits[0]].loc[:, 'timestampMonth'].unique())

Super simplistic input data

In [ ]:
# %%time
# procs = [Categorify, FillMissing, Normalize]
# to = TabularPandas(X, procs, ['meter'],
#                    [], y_names='meter_reading', splits=splits)

All input data

In [ ]:
%%time
procs = [Categorify, FillMissing, Normalize]
to = TabularPandas(X, procs, categorical,
                   continuous, 
                   y_names='meter_reading', splits=splits)

train_bs = 256*4
val_bs = 256*4
dls = to.dataloaders(bs=train_bs, val_bs=val_bs)

In [ ]:
%%time
if predict_test:
    test_dl = dls.test_dl(X_test, bs=1024*4)

In [ ]:
to.train.xs

In [ ]:
to.train.ys

## Modelling

In [ ]:
def hist_plot_preds(y_valid_pred, y_test_pred):
    res = pd.concat(
        (
            pd.DataFrame({
                'y': y_valid_pred, 
                'set': ['valid']*len(y_valid_pred)
            }),
            pd.DataFrame({
                'y':y_test_pred, 
                'set': ['test']*len(y_test_pred)
            })
        ),
        ignore_index=True
    )

    return px.histogram(res, x='y', color='set', marginal='box',
                        barmode='overlay', histnorm='probability density')

### super simple base model

In [ ]:
class SimpleEstimator:
    cols = []
    def fit(self, X, y):
        self.cols = [col for col in ['building_id', 'meter'] if col in X.columns]
        tmp = pd.concat((X.loc[:,self.cols], to.train.ys), axis=1)
        self.consts = tmp.groupby(self.cols)['meter_reading'].describe().to_dict()
    def predict(self, X, quantity:str='mean'):
        return [self.consts[quantity][tuple(row[self.cols].values)] for i,row in X.iterrows()]

In [ ]:
se = SimpleEstimator()

In [ ]:
%%time
se.fit(to.train.xs, to.train.ys)

In [ ]:
%%time
y_valid_pred = se.predict(to.valid.xs, quantity='mean')

In [ ]:
y_true = to.valid.ys.values.ravel()
y_pred = y_valid_pred
assert y_true.shape[0] == len(y_pred)
evaluate_torch(torch.from_numpy(y_true), 
               torch.Tensor(y_pred))

In [ ]:
%%time
if predict_test:
    y_test_pred = se.predict(test_dl.xs)
    display(y_test_pred[:5])

In [ ]:
%%time
if predict_test:
    _y_train = np.random.choice(y_valid_pred, size=5000)
    _y_test = np.random.choice(y_test_pred, size=5000)

    hist_plot_preds(_y_train, _y_test)

In [ ]:
%%time
if predict_test:
    y_test_pred = torch.Tensor(y_test_pred)

### Modelling with sklearn

In [ ]:
m = linear_model.LinearRegression()

In [ ]:
m = ensemble.RandomForestRegressor(n_estimators=100, max_features=.75, criterion='mse')

In [ ]:
%%time
m.fit(to.train.xs, to.train.ys.values.ravel())

In [ ]:
y_valid_pred = m.predict(to.valid.xs.values)
y_valid_pred[:5]

In [ ]:
y_true = to.valid.ys.values.ravel()
y_pred = y_valid_pred
assert y_true.shape == y_pred.shape
nb_score = evaluate_torch(torch.from_numpy(y_true), 
                          torch.from_numpy(y_pred))
nb_score.item()

**Plot boldly wrong time series**

Get errors by `building_id` and `meter`

In [ ]:
#export
class BoldlyWrongTimeseries:
    def __init__(self, xs, y_true, y_pred):
        self.df = xs.loc[:,['meter', 'building_id', 'timestampElapsed']].copy()
#         self.get_predictions(model, xs)
        self.df['y_true'] = y_true
        self.df['y_pred'] = y_pred
        self.compute_misses(y_true)

#     def get_predictions(self, model, xs):
#         self.df['y_pred'] = m.predict(xs)

    def compute_misses(self, y_true):
        fun = lambda x: np.mean(x**2)
        self.miss = (self.df.assign(difference=lambda x: x['y_pred']-x['y_true'])
                     .groupby(['meter', 'building_id'])
                     .agg(loss=pd.NamedAgg(column='difference', aggfunc=fun))
                     .sort_values('loss'))

In [ ]:
%%time
bwt = BoldlyWrongTimeseries(to.valid.xs, to.valid.ys, y_pred)

Adding something to plot

In [ ]:
#export
def plot_boldly_wrong(self, nth_last:int=None,
                      meter:int=None, bid:int=None):
    assert (meter is not None and bid is not None) or (nth_last is not None)
    if nth_last is not None:
        ix = self.miss.iloc[[nth_last],:]
        meter = ix.index[0][0]
        bid = ix.index[0][1]
        loss = ix["loss"].values[0]
    else:
        ix = self.miss.xs((meter, bid))
        loss = ix.values[0]
        
    df_plot = self.df.loc[(self.df['meter']==meter) & (self.df['building_id']==bid)]
    df_plot = pd.concat((
        df_plot[['timestampElapsed', 'y_true']].rename(columns={'y_true':'y'}).assign(label='true'),
        df_plot[['timestampElapsed', 'y_pred']].rename(columns={'y_pred':'y'}).assign(label='pred'))
    )
    return df_plot.plot(kind='scatter', x='timestampElapsed', 
                        y='y', color='label', 
                        title=f'pos {nth_last}: meter = {meter}, building_id = {bid}<br>loss = {loss:.3f}')
    
    
BoldlyWrongTimeseries.plot_boldly_wrong = plot_boldly_wrong

In [ ]:
bwt.plot_boldly_wrong(nth_last=-1)

In [ ]:
bwt.plot_boldly_wrong(meter=1, bid=100)

Adding widgets

In [ ]:
#export
def init_widgets(self):
    self.int_txt_loss = widgets.IntText(min=-len(self.miss), max=len(self.miss),
                                   description='Position')
    self.int_txt_meter = widgets.IntText(min=self.df['meter'].min(), max=self.df['meter'].max(),
                                   description='Meter')
    self.int_txt_bid = widgets.IntText(min=self.df['building_id'].min(), max=self.df['building_id'].max(),
                                   description='building id')
    self.run_btn = widgets.Button(description='plot')
    self.switch_btn = widgets.Checkbox(description='Loss-based', value=True)
    self.run_btn.on_click(self.click_boldly_wrong)
    self.out_wdg = widgets.Output()
    
def run_boldly(self):
    if not hasattr(self, 'switch_btn'):
        self.init_widgets()
    return widgets.VBox([self.switch_btn, self.int_txt_loss, 
                         self.int_txt_meter, self.int_txt_bid, 
                         self.run_btn, self.out_wdg])

def click_boldly_wrong(self, change):
    self.out_wdg.clear_output()
    nth_last = None if self.switch_btn.value == False else self.int_txt_loss.value
    meter = None if self.switch_btn.value == True else self.int_txt_meter.value
    bid = None if self.switch_btn.value == True else self.int_txt_bid.value
    with self.out_wdg:
        print(f'nth_last {nth_last} meter {meter} bid {bid}')
        try:
            self.plot_boldly_wrong(nth_last=nth_last, meter=meter, bid=bid).show()
        except:
            raise ValueError(f'nth_last {nth_last} meter {meter} bid {bid} not a valid combination! Likely due to missing meter/bid combination')

BoldlyWrongTimeseries.init_widgets = init_widgets
BoldlyWrongTimeseries.click_boldly_wrong = click_boldly_wrong
BoldlyWrongTimeseries.run_boldly = run_boldly

In [ ]:
bwt.run_boldly()

Sanity check predictions `dep_var` distribution vaid vs test

In [ ]:
%%time
if predict_test:
    y_test_pred = m.predict(test_dl.xs)
    display(y_test_pred[:5])

In [ ]:
%%time
if predict_test:
    _y_train = np.random.choice(y_valid_pred, size=5000)
    _y_test = np.random.choice(y_test_pred, size=5000)

    hist_plot_preds(_y_train, _y_test)

In [ ]:
%%time
if predict_test:
    y_test_pred = torch.Tensor(y_test_pred)

Finding:
- values in the range of 90. this is way to large, but the values predicted for the validation set are okay. what is different between the prediction over the validation set and the test set?

### Modelling with fastai

In [ ]:
y_range = [np.min([to.train.ys.values.min(), to.valid.ys.values.min()]),
           np.max([to.train.ys.values.max(), to.valid.ys.values.max()]),]
y_range

In [ ]:
y_range = [0,
           np.max([to.train.ys.values.max(), to.valid.ys.values.max()]),]
y_range

In [ ]:
# y_range = [to.train.ys.values.min(),
#            to.train.ys.values.max()]
# y_range

In [ ]:
learn = tabular_learner(dls, y_range=y_range, layers=[500,250],
                        n_out=1, loss_func=evaluate_torch)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, 1e-2)

In [ ]:
y_valid_pred, y_valid_true = learn.get_preds()
y_valid_pred[:5], y_valid_true[:5]

In [ ]:
nb_score = evaluate_torch(y_valid_pred, y_valid_true)
nb_score.item()

In [ ]:
# clone numpy ravel
cnr = lambda x: x.clone().numpy().ravel()

In [ ]:
%%time
bwt_tl = BoldlyWrongTimeseries(to.valid.xs, cnr(y_valid_true), cnr(y_valid_pred))

In [ ]:
bwt_tl.run_boldly()

In [ ]:
bwt.run_boldly()

In [ ]:
if predict_test:
    y_test_pred, _ = learn.get_preds(dl=test_dl)
    display(y_test_pred[:5])

In [ ]:
%%time
if predict_test:
    _y_valid = y_valid_pred.clone().numpy().ravel()
    _y_valid = np.random.choice(_y_valid, size=5000)

    _y_test = y_test_pred.clone().numpy().ravel()
    _y_test = np.random.choice(_y_test, size=5000)

    hist_plot_preds(_y_valid, _y_test)

## Transforming back and storing in submission format

In [ ]:
if predict_test:
    y_test_pred_original = torch.exp(y_test_pred) - 1
    displa(y_test_pred_original[:5])

In [ ]:
if predict_test:
    y_out = pd.DataFrame(y_test_pred_original.clone().numpy(),
                         columns=['meter_reading'])
    y_out.index.rename('row_id', inplace=True)
    display(y_out.head())

In [ ]:
if predict_test:
    assert len(y_out) == 41697600

In [ ]:
%%time
if predict_test:
    y_out.to_csv('test_submission_randomforest.csv')

In [ ]:
%%time
if predict_test:
    y_out.to_csv('test_submission_tabularlearner.csv')

`kaggle competitions submit -c ashrae-energy-prediction -f submission.csv -m "Message"`

In [ ]:
# !kaggle competitions submit -c ashrae-energy-prediction -f test_submission_randomforest.csv -m f"50 obs per bid - randomforest with filtered outliers nb score {nb_score}"

In [ ]:
# !kaggle competitions submit -c ashrae-energy-prediction -f test_submission_tabularlearner.csv -m f"1000 obs per bid - tabularlearner with bs=256 and filtered outliers & 0s and bid&sid nb score {nb_score}"

**submission scores**

random forest:
- 5 obs per building ID, .75 max_features, 100 estimators: 
    - nb score = 2.37
    - kaggle score = 1.68 / 1.86
    
tabular learner:
- 5 obs per building ID, layers=[500,250], lr = 2e-3: 
    - nb score = 1.55
    - kaggle score = 1.8 / 2.13
- 5 obs per building ID, layers=[500,250], second run with lr = 1e-3: 
    - nb score = 1.57
    - kaggle score = 1.846 / 2.13
- 50 obs per building ID, layers=[500,250], 2 rounds: 
    - nb score = 1.39
    - kaggle score = 1.722 / 2.51
- 50 obs per building ID, layers=[500,250], 2 rounds: 
    - nb score = 1.34
    - kaggle score = 1.641 / 2.266
- 50 obs per building ID, layers=[500,250], 2 rounds, bs=256: 
    - nb score = 1.32
    - kaggle score = 1.643 / 1.926
- 500 obs per building ID, layers=[500,250], 3 rounds: 
    - nb score = 1.19
    - kaggle score = 1.62 / 2.55

Finding:
- nb scores are lower than the kaggle scores
- random forest seems to have public and private score closer to each other than tabular learner

**randomly splitting**
    
Finding (modified target values, all info = info except time):
- Linear:
    - meter only @100k: 2.1
    - all info minus time @100k: 2.3
    - all info incl time @100k: 2.32
    - all info incl time + ids @100k: 2.32
- RandomForest:
    - meter only @100k: 2.2
    - all info minus time @100k: 2.7
    - all info incl time @100k: 2.74
    - all info incl time + ids @100k: 2.82
- tabular_learner:
    - meter only @100k: 2.1
    - all info minus time @100k: 1.56
    - all info incl time @100k: 1.52
    - all info incl time + ids @100k: 0.96
    
**splitting along time**
Finding:
- Linear:
    - meter only @100k: 2.1
    - all info minus time @100K: 2.2
    - all info incl time @100k: 2.3
    - all info incl time + ids @100k: 2.29
- RandomForest:
    - meter only @100k: 2.1
    - all info minus time @100K: 2.7
    - all info incl time @100k: 2.52
    - all info incl time + ids @100k: 2.62
- tabular_learner:
    - meter only @100k: 2.06
    - all info minus time @100K: 1.62
    - all info incl time @100k: 1.62
    - all info incl time + ids @100k: 1.31